# Тема 2. Деревья принятия решений

## Agenda
В данном разделе вы познакомитесь с основными библиотеками python для анализа данных: pandas, matplotlib, sklearn и попробуете на практике провести анализ и сделать выводы по исходным данным.  
Целью данной лабораторной работы изучить принцип работы модели деревьев принятия решений, ее возможности.  
В качестве исходных данных был выбран датасет титаника (https://www.kaggle.com/c/titanic/data). 

## Дерево принятий решений
Дерево принятий решений - модель реализующая дерево, разделяющее каждый из объектов по заданным признакам. В ветках (ребрах) данного дерева находится условие разделения, а в листовых вершинах класс, к которому данное дерево относит объект. Пример такого дерева представлен на рисунке ниже

![asda](./images/treeExample.png "Title")

In [1]:
# Загрузим необходимы библиотеки
import pandas as pd
pd.set_option('display.max.columns', 100)
import matplotlib.pyplot as plt
import seaborn as sns

# Импортируем класс деревьев
from sklearn.tree import DecisionTreeClassifier
# И Функцию разделения на тестовую и обучающую выборку
from sklearn.model_selection import train_test_split

# Библиотека с математическими операциями над матрицами, 
# которая лежит в основе pandas
import numpy as np

# Для возможности вывода картинок прямо в тетрадь
%matplotlib inline 

In [2]:
# Загрузка дата сета
data = pd.read_csv("datasets/titanic_train.csv", index_col="PassengerId")

# Вывод на экран первых 5 записей
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Создадим дубликат датасета без стоблца Cabin и удалим записи содержащие пустые ячейки
data_full = data.drop("Cabin", axis=1).dropna()

In [4]:
# Заменяем строку "male" на 0, "female" на 1
data_full["Gender"] = data_full["Sex"].map({"male": 0, "female": 1})
data_full["Gender"].head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Gender, dtype: int64

In [5]:
data_full.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Gender
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,1
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,1
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,1
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,0


In [6]:
# Выделим отдельно стобцы на основе которых будем строить модель
X = data_full[["Age", "Gender"]]
# Выделим целевую переменную
y = data_full.Survived

In [7]:
# Разделим выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, random_state=40,
                                                   shuffle=True)

Для начала попробуем без настройки гиперпараметров обучить дерево

In [8]:
# Создадим модель и установим фиксированный random_state для воспроизводимости результатов
model = DecisionTreeClassifier(random_state=42)

In [9]:
# Обучим модель
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [10]:
# Проверим долю правильных ответов нашей модели на тестовой выборке
acc = np.sum(model.predict(X_test) == y_test) / X_test.shape[0]
print("Доля правильных ответов = {}".format(acc))

Доля правильных ответов = 0.7132867132867133


Как мы видим с 71% точностью наша модель правильно определяет выживаемость того или иного пассажира титаника

Попробуем улучшить нашу модель. Добавим 3 новых признака: первый класс, второй класс и третий класс. Эти данные можно взять из столбца "Pclass".
Для разбития качественного признака в несколько бинарных сущесвует метод **pd.get_dummies**

In [11]:
data_full = pd.concat([data_full, pd.get_dummies(data_full["Pclass"], prefix="Pclass")], axis=1)
data_full.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Gender,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,0,0,0,1
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,1,1,0,0
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,1,0,0,1
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,1,1,0,0
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,0,0,0,1


In [12]:
# Выделим отдельно стобцы на основе которых будем строить модель
selected_cols = ["Pclass_1", "Pclass_2", "Pclass_3", "Age", "Gender"]
X = data_full[selected_cols]
# Выделим целевую переменную
y = data_full.Survived

In [13]:
# Разделим выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, random_state=40,
                                                   shuffle=True)

In [14]:
# Создадим модель и установим фиксированный random_state для воспроизводимости результатов
model = DecisionTreeClassifier(random_state=42)
# Обучим модель
model.fit(X_train, y_train)
# Проверим долю правильных ответов нашей модели на тестовой выборке
acc = np.sum(model.predict(X_test) == y_test) / X_test.shape[0]
print("Доля правильных ответов = {}".format(acc))

Доля правильных ответов = 0.7762237762237763


In [15]:
# Выведем признаки и их важности в модели
for i, col_name in enumerate(selected_cols):
    print("{} = {}".format(col_name, model.feature_importances_[i]))

Pclass_1 = 0.03848726745272861
Pclass_2 = 0.011238558869084689
Pclass_3 = 0.13323160379773744
Age = 0.4079201756465267
Gender = 0.40912239423392255


Как видно, наибольшее влияние на выживаемость влияет возраст и класс билета. Теперь попробуем подобрать гиперпараметры, чтобы доля правильных ответов на тестовой выборке повысилась. Основным гиперпараметром дерева является его глубина. При большой глубине дерева, оно может переобучиться и выдавать плохие результаты на тестовой выборке, но отличные на обучающей. 

In [16]:
# Попробуем построить деревья с максимальной глубиной от 2 до 15
for depth in range(2, 15):
    # Создадим модель и установим фиксированный random_state для воспроизводимости результатов
    model = DecisionTreeClassifier(random_state=42, max_depth=depth)
    # Обучим модель
    model.fit(X_train, y_train)
    # Проверим долю правильных ответов нашей модели на тестовой выборке
    acc = np.sum(model.predict(X_test) == y_test) / X_test.shape[0]
    print("Доля правильных ответов = {} при глубине равной = {}".format(acc, depth))

Доля правильных ответов = 0.7622377622377622 при глубине равной = 2
Доля правильных ответов = 0.7762237762237763 при глубине равной = 3
Доля правильных ответов = 0.7762237762237763 при глубине равной = 4
Доля правильных ответов = 0.7622377622377622 при глубине равной = 5
Доля правильных ответов = 0.7762237762237763 при глубине равной = 6
Доля правильных ответов = 0.7762237762237763 при глубине равной = 7
Доля правильных ответов = 0.7762237762237763 при глубине равной = 8
Доля правильных ответов = 0.7832167832167832 при глубине равной = 9
Доля правильных ответов = 0.7692307692307693 при глубине равной = 10
Доля правильных ответов = 0.7692307692307693 при глубине равной = 11
Доля правильных ответов = 0.7762237762237763 при глубине равной = 12
Доля правильных ответов = 0.7762237762237763 при глубине равной = 13
Доля правильных ответов = 0.7762237762237763 при глубине равной = 14


Как видно, при глубине равно 9 достигается максимальная доля правильных ответов равной 0.783. Это значение является наиболее подходящим для данной модели